In [1]:
%pip install statsbombpy

  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.27m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [typing_extensions]
  Attempting uninstall: attrs8;5;237m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [typing_extensions]
    Found existing installation: attrs 24.3.08;5;237m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [typing_extensions]
    Uninstalling attrs-24.3.0:;5;237m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [typing_extensions]
      Successfully uninstalled attrs-24.3.0━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [typing_extensions]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [statsbombpy] 4/8 [cattrs]nsions]
Note: you may need to restart the kernel to use updated packages.


In [32]:
from statsbombpy import sb

sb.competitions()

/opt/anaconda3/lib/python3.13/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-09-28T20:46:38.893391,2025-07-06T04:26:07.636270,2025-07-06T04:26:07.636270,2024-09-28T20:46:38.893391
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-09-28T01:57:35.846538,None,None,2024-09-28T01:57:35.846538
3,16,4,Europe,Champions League,male,False,False,2018/2019,2025-05-08T15:10:50.835274,2021-06-13T16:17:31.694,None,2025-05-08T15:10:50.835274
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882
...,...,...,...,...,...,...,...,...,...,...,...,...
70,35,75,Europe,UEFA Europa League,male,False,False,1988/1989,2024-02-12T14:45:05.702250,2021-06-13T16:17:31.694,None,2024-02-12T14:45:05.702250
71,53,315,Europe,UEFA Women's Euro,female,False,True,2025,2025-07-28T14:19:20.467348,2025-07-29T16:03:07.355174,2025-07-29T16:03:07.355174,2025-07-28T14:19:20.467348
72,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2024-02-13T13:27:17.178263,2024-02-13T13:30:52.820588,2024-02-13T13:30:52.820588,2024-02-13T13:27:17.178263
73,72,107,International,Women's World Cup,female,False,True,2023,2025-07-14T10:07:06.620906,2025-07-14T10:10:27.224586,2025-07-14T10:10:27.224586,2025-07-14T10:07:06.620906


In [33]:
sb.matches(competition_id= 43, season_id=3)

/opt/anaconda3/lib/python3.13/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,7585,2018-07-03,20:00:00.000,International - FIFA World Cup,2018,Colombia,England,1,1,available,...,2021-06-13T16:17:31.694,4,Round of 16,Otkritie Bank Arena,Mark Geiger,José Néstor Pekerman,Gareth Southgate,1.0.2,None,None
1,7570,2018-06-28,20:00:00.000,International - FIFA World Cup,2018,England,Belgium,0,1,available,...,2021-06-13T16:17:31.694,3,Group Stage,Stadion Kaliningrad,Damir Skomina,Gareth Southgate,Roberto Martínez Montoliú,1.0.2,None,None
2,7586,2018-07-03,16:00:00.000,International - FIFA World Cup,2018,Sweden,Switzerland,1,0,available,...,2021-06-13T16:17:31.694,4,Round of 16,Saint-Petersburg Stadium,Damir Skomina,Jan Olof Andersson,Vladimir Petković,1.0.2,None,None
3,7557,2018-06-25,20:00:00.000,International - FIFA World Cup,2018,Iran,Portugal,1,1,available,...,2021-06-13T16:17:31.694,3,Group Stage,Mordovia Arena,Enrique Cáceres,Carlos Manuel Brito Leal Queiróz,Fernando Manuel Fernandes da Costa Santos,1.0.2,None,None
4,7542,2018-06-20,14:00:00.000,International - FIFA World Cup,2018,Portugal,Morocco,1,0,available,...,2021-06-13T16:17:31.694,2,Group Stage,Stadion Luzhniki,Mark Geiger,Fernando Manuel Fernandes da Costa Santos,Hervé Renard,1.0.2,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,7540,2018-06-19,20:00:00.000,International - FIFA World Cup,2018,Russia,Egypt,3,1,available,...,2021-06-13T16:17:31.694,2,Group Stage,Saint-Petersburg Stadium,Enrique Cáceres,Stanislav Cherchesov,Héctor Raúl Cúper,1.0.2,None,None
60,8652,2018-07-07,20:00:00.000,International - FIFA World Cup,2018,Russia,Croatia,2,2,available,...,2021-06-13T16:17:31.694,5,Quarter-finals,\tOlimpiyskiy Stadion Fisht,Sandro Ricci,Stanislav Cherchesov,Zlatko Dalić,1.0.2,None,None
61,7563,2018-06-26,16:00:00.000,International - FIFA World Cup,2018,Denmark,France,0,0,available,...,2021-06-13T16:17:31.694,3,Group Stage,Stadion Luzhniki,Sandro Ricci,Åge Fridtjof Hareide,Didier Deschamps,1.0.2,None,None
62,7556,2018-06-24,17:00:00.000,International - FIFA World Cup,2018,Japan,Senegal,2,2,available,...,2021-06-13T16:17:31.694,2,Group Stage,\tEkaterinburg Arena,Gianluca Rocchi,Akira Nishino,Aliou Cissé,1.0.2,None,None


In [34]:
events = sb.events(match_id=7585)

/opt/anaconda3/lib/python3.13/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [35]:
events.keys()

Index(['50_50', 'bad_behaviour_card', 'ball_receipt_outcome',
       'ball_recovery_recovery_failure', 'block_deflection', 'block_offensive',
       'carry_end_location', 'clearance_aerial_won', 'counterpress',
       'dribble_nutmeg', 'dribble_outcome', 'duel_outcome', 'duel_type',
       'duration', 'foul_committed_advantage', 'foul_committed_card',
       'foul_committed_penalty', 'foul_committed_type', 'foul_won_advantage',
       'foul_won_defensive', 'foul_won_penalty', 'goalkeeper_body_part',
       'goalkeeper_end_location', 'goalkeeper_outcome', 'goalkeeper_position',
       'goalkeeper_technique', 'goalkeeper_type', 'id', 'index',
       'interception_outcome', 'location', 'match_id', 'minute',
       'pass_aerial_won', 'pass_angle', 'pass_assisted_shot_id',
       'pass_backheel', 'pass_body_part', 'pass_cross', 'pass_deflected',
       'pass_end_location', 'pass_goal_assist', 'pass_height', 'pass_length',
       'pass_outcome', 'pass_recipient', 'pass_recipient_id',
       

In [36]:
events = events[['team', 'tactics', 'player', 'type', 'location', 'minute', 'pass_end_location']]

In [39]:
events.head(100)

,team,tactics,player,type,location,minute,pass_end_location
0,Colombia,"{'formation': 433, 'lineup': [{'player': {'id'...",NaN,Starting XI,NaN,0,NaN
1,England,"{'formation': 352, 'lineup': [{'player': {'id'...",NaN,Starting XI,NaN,0,NaN
2,England,NaN,NaN,Half Start,NaN,0,NaN
3,Colombia,NaN,NaN,Half Start,NaN,0,NaN
4,Colombia,NaN,NaN,Half Start,NaN,45,NaN
...,...,...,...,...,...,...,...
95,England,NaN,Bamidele Alli,Pass,"[83.0, 3.0]",8,"[83.0, 3.0]"
96,Colombia,NaN,Carlos Alberto Sánchez Moreno,Pass,"[28.0, 66.0]",8,"[23.0, 50.0]"
97,Colombia,NaN,Davinson Sánchez Mina,Pass,"[23.0, 50.0]",8,"[48.0, 40.0]"
98,Colombia,NaN,Juan Guillermo Cuadrado Bello,Pass,"[48.0, 40.0]",8,"[39.0, 39.0]"


In [41]:
events

,team,tactics,player,type,location,minute,pass_end_location
0,Colombia,"{'formation': 433, 'lineup': [{'player': {'id'...",NaN,Starting XI,NaN,0,NaN
1,England,"{'formation': 352, 'lineup': [{'player': {'id'...",NaN,Starting XI,NaN,0,NaN
2,England,NaN,NaN,Half Start,NaN,0,NaN
3,Colombia,NaN,NaN,Half Start,NaN,0,NaN
4,Colombia,NaN,NaN,Half Start,NaN,45,NaN
...,...,...,...,...,...,...,...
4014,England,NaN,Kyle Walker,Substitution,NaN,112,NaN
4015,Colombia,NaN,Santiago Arias Naranjo,Substitution,NaN,115,NaN
4016,Colombia,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,Tactical Shift,NaN,61,NaN
4017,Colombia,NaN,NaN,Camera off,NaN,76,NaN
